# Presentación

👋 Este cuaderno colab es un recurso generado por Zoraida Callejas y David Griol para la asignatura de [Tecnologías del habla y del Lenguaje Natural](https://masteres.ugr.es/desarrollo-software/docencia/plan-estudios/guia-docente/M52/56/5/23) del Máster de Desarrollo de Software de la Universidad de Granada. La primera parte del cuaderno es una adaptación del notebook compartido por [Vaibhav Srivastav](https://colab.research.google.com/github/Vaibhavs10/insanely-fast-whisper/blob/main/infer_faster_whisper_large_v2.ipynb).

Siguiendo el cuaderno aprenderás a:

*   Utilizar **modelos pre-entrenados** basados en *deep learning* para el reconocimiento del habla.
*   Subir, descargar, grabar, reproducir y recortar audios.
*   Visualizar audios.

Puedes ir siguiéndolo y ejecutando cada celda de código secuencialmente.

# Instalaciones necesarias

El ejemplo que vas a seguir se basa en el modelo <a href="https://huggingface.co/openai/whisper-large-v3">whisper-large-v3</a> de OpenAI. Como se quiere evitar tener que usar grandes recursos CPU/GPU (🚫💸) , se propone utilizar una reimplementación de Whisper mucho más rápida llamada <a href="https://pypi.org/project/faster-whisper/">**Faster Whisper**</a>.

En primer lugar instala faster-whisper:

In [ ]:
!pip install --upgrade -q faster-whisper ipython-autotime

En aplicaciones que requieren reconocer el habla, el tiempo de ejecución suele ser clave. Para saber lo que tarda en ejecutarse cada celda de código de este cuaderno, carga la extensión "autotime". Una vez activada, sabrás cuánto ha tardado en ejecutarse cada celda.



In [ ]:
%load_ext autotime

# Descarga un archivo de audio para probar el modelo

Prueba en primer lugar el audio del podcast [Lex interviewing Sam Altman](https://www.youtube.com/watch?v=L_Guz73e6fw&t=8s) que está subido en hugginface [aquí](https://huggingface.co/datasets/reach-vb/random-audios/blob/main/sam_altman_lex_podcast_367.flac).

In [ ]:
!wget https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/sam_altman_lex_podcast_367.flac

# Reconoce habla utilizando el modelo pre-entrenado

Crea una instancia de la clase **WhisperModel** del módulo "faster_whisper" (ver la documentación en [github](https://github.com/SYSTRAN/faster-whisper/blob/master/faster_whisper/transcribe.py)) utilizando los siguientes parámetros en el constructor:
* El tamaño del modelo será "large-v3" (posibles valores: tiny, tiny.en, base, base.en, small, small.en, medium, medium.en, large-v1, large-v2, large-v3, o large).
* El dispositivo donde se ejecutará será una GPU usando CUDA (otros posibles valores: "cpu", "cuda", "auto").
* Se utilizará una representación de punto flotante de 16 bits para los cálculos (otros posibles valores documentados [aquí](https://opennmt.net/CTranslate2/quantization.html)).

Puedes probar a jugar con distintas combinaciones y ver su repercusión los tiempos de reconocimiento.

In [ ]:
# Importamos la clase WhisperModel
from faster_whisper import WhisperModel

# Generamos una instacia denominada mi_modelo
mi_modelo = WhisperModel(model_size_or_path="large-v3", device="auto", compute_type="float16")

Ejecuta el método **transcribe** para transcribir el archivo de audio. El método recibe como entrada:
* La ruta al archivo de audio.
* Beam size. Durante el proceso de decodificación, el modelo genera una secuencia de posibles tokens (palabras o incluso subpalabras) que puedan venir a continuación. El algoritmo *beam search* explora estas posibilidades para encontrar la secuencia más probable de tokens. El *beam size* determina cuántas secuencias posibles utiliza el algoritmo de decodificación en cada paso.

y genera como salida una dupla con:
* Los segmentos transcritos.
* Información sobre el reconocimiento.

Puedes probar más parámetros de entrada como:
* language: el idioma ("es", "en"...). Si no se indica lo reconoce con los primeros 30 segundos del audio.
* Ver más en [github](https://github.com/SYSTRAN/faster-whisper/blob/master/faster_whisper/transcribe.py).


🐌 Ten paciencia, ejecutar esta celda de código puede tardar un rato.




In [ ]:
# Ejecutamos el método transcribe (OJO: Puede tardar un rato, según el tamaño del fichero de audio)
segmentos, info = mi_modelo.transcribe(audio="Audio Explicativo.mp3", beam_size=1)

¡Ya puedes comprobar los resultados de la transcripción! 🥳


Por ejemplo, la información que has recibido en la variable **info**:
* language: idioma detectado.
* language_probability: con qué probabilidad el modelo considera que se trata de ese idioma en particular.
* all_language_probs: probabilidad de todos los idiomas que es capaz de reconocer.
* duration: duración del audio.
* duration_after_vad: duración después de centrarse únicamente en los segementos del audio que contienen habla (en contraprosición a los silencios o ruidos de fondo).



In [ ]:
# Imprimimos el idioma detectado (durante los primeros 30s del audio) y con qué probabilidad se trata de ese idioma (con 2 decimales)
print("El idioma detectado es: '%s', con probabilidad %.2f" % (info.language, info.language_probability))

# Imprimimos la probabilidad asociada a cada idioma (si esta es mayor de 0.000001)
for idioma, prob in info.all_language_probs:
    if prob > 0.000001:
        print(f"{idioma}, {prob:f}")

Lo verdaderamente interesante está en los segmentos. La señal de audio se divide en segmentos consecutivos basados en las decisiones del sistema VAD (voice activity detection). Los segmentos con habla se mantienen y procesan. Ahora mismo, tienes almacenados estos segmentos en la variable **segmentos**, junto con información como la marca de tiempo de inicio y fin y la transcripción del habla contenida en cada segmento. Para imprimirlos ejecuta:

In [ ]:
for segmento in segmentos:
    print("[%.2fs -> %.2fs] %s" % (segmento.start, segmento.end, segmento.text))
    if(segmento.start > 300):
       break # Poner breaks en un bucle es muuuuuy mala práctica de programación 🫣,
              # pero nos interesa ver cómo ha funcionado más que mostrar la larga lista de segmentos completa.
              # De todas formas, aquí mandas tú: si quieres ver la transcripción completa borra el condicional

**¡IMPRESIONANTE!**

Compara el texto con el audio original, que puedes escuchar [aquí](https://cdn-lfs.huggingface.co/repos/96/e4/96e4f69cd112b019dd764318570e47e5fe96de53d8c32a99d745e72d9086e355/b2fd593ce144a8d904cf49a4ed77ed06eb50644a053dddd280c81a3ef94fb60e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sam_altman_lex_podcast_367.flac%3B+filename%3D%22sam_altman_lex_podcast_367.flac%22%3B&response-content-type=audio%2Fx-flac&Expires=1709058168&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTA1ODE2OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85Ni9lNC85NmU0ZjY5Y2QxMTJiMDE5ZGQ3NjQzMTg1NzBlNDdlNWZlOTZkZTUzZDhjMzJhOTlkNzQ1ZTcyZDkwODZlMzU1L2IyZmQ1OTNjZTE0NGE4ZDkwNGNmNDlhNGVkNzdlZDA2ZWI1MDY0NGEwNTNkZGRkMjgwYzgxYTNlZjk0ZmI2MGU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=QCSGtP08p93enH%7E3lRIhMO6zDqQVfuUW96AAx9%7E%7ErKbGVZV5-13UzaKSOOfu7H4N3JlVhRF-pV4q4lb8Vx1Doc2ZSLjd-JwFKl6VQsPOuHF5iilftzs2g1PjR5mfDN31Bqgcc1Sho14yzIlaPhEh2xXjk-BnYVZtFdkr6RAP9PrWSMkadJEeNgt78STVmmeHMNvn1q1ds2U9m8AUj8oeeKPNGTsqM623kTdshogo9U0HWhHpY4h6hbtpYxjUzL%7EB2WC5I6YK7YlJN9C3VZYlTvU6Pf-zvLc1lJ-yQRDu7J1j7uJ2JqlucvCtuaRfOM4KuRz%7Elcfib5NU%7EZLwdmVaNQ__&Key-Pair-Id=KVTP0A1DKRTAX)...

El tiempo de transcripción para esta gran calidad de resultado es increíble, ¿verdad? 🤯

Puedes reducirlo incluso más jugando con los parámetros con los que creaste la instancia de WhisperModel, por ejemplo usando "int8_float16" en el parámetro compute_type. Haz la prueba...

# Juega un poco más 🛝

A continuación se proponen algunas pruebas para trabajar en clase. Éstas están menos guiadas, para que tengas más autonomía al plantear cómo resolverlas; pero no te preocupes: son sencillas y se dan algunas pistas...

## Replica lo anterior subiendo tu propio archivo de audio

Prueba a transcribir tus propios audios mp3. Puede ser un buen momento para ver cómo funciona en español. Lo ideal para no tener tiempos de espera muy largos es que el ejemplo que utilices dure aproximadamente uno o dos minutos.

No te cortes copiando, pegando y reutilizando celdas de código de la parte anterior. Este cuaderno es tuyo para que experimentes todo lo que quieras  🙂

In [ ]:
# Este trocito de código hace que salga un botón examinar y puedas subir un fichero.
# Para usarlo después, la ruta será el nombre del archivo, por ejemplo MiAudio.mp3

# En su lugar también puedes usar el icono de la carpeta del menú de la izquierda
# y trabajar con la interfaz gráfica

from google.colab import files

uploaded = files.upload()

## Prueba con un mp3 largo que esté colgado en una URL y tengas que recortar

In [ ]:
#Aquí tienes un ejemplo con un podcast de RNE

!pip install requests pydub

import requests
from pydub import AudioSegment

# Función para descargar el mp3 de una url
def download_mp3(url, archivoCompleto):
    respuesta = requests.get(url)
    with open(archivoCompleto, 'wb') as f:
        f.write(respuesta.content)

# Función para cortar el audio entre dos marcas de tiempo
def cut_audio(input_path, output_path, start_time, end_time):
    audio = AudioSegment.from_file(input_path, format="mp3")
    corte = audio[start_time:end_time]
    corte.export(output_path, format="mp3")

# Descarga y corta. Aquí estamos cortando entre el segundo 10 y el 90
url_podcast = 'https://rtve-mediavod-lote3.rtve.es/resources/TE_SYSISIS/mp3/3/1/1708708893513.mp3'
download_mp3(url_podcast, 'miaudio.mp3')
cut_audio('miaudio.mp3', 'miaudio_cortado.mp3', start_time=10000, end_time=20000)

## Prueba ahora a grabar un audio desde tu micrófono

In [ ]:
# Este trozo de código está basado en: http://blog.syntheticspeech.de/2021/03/22/recording-and-transcribing-a-speech-sample-on-google-colab/

from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def graba(ficheroAudio, segundos):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (segundos*1000))
  b = b64decode(s.split(',')[1])
  with open(ficheroAudio,'wb') as f:
    f.write(b)
  return "Audio grabado en "+ficheroAudio

In [ ]:
# Puedes usar la función "graba" como se indica a continuación.
# ⚠️ En cuanto ejecutes se pondrá a grabar durante los segundos que hayas indicado (en el ejemplo 5).
# ⚠️ Recuerda dar permisos en tu navegador para usar el micrófono.
import os
graba('./migrabacion.wav', 5)

In [ ]:
# Puedes reproducir el audio:
from IPython.display import Audio
Audio('migrabacion.wav')

## Has probado con diferentes audios... prueba ahora diferentes modelos

Hemos probado un modelo pre-entrenado basado en Whisper.

**SpeechBrain** es un toolkit de propósito general sobre tecnologías del habla que provee 100 modelos pre-entrenados de reconocimiento de habla que puedes probar. Te recomendamos que visites su [página web](https://speechbrain.github.io/), pues está repleta de recursos sobre procesamiento del habla.

In [ ]:
# Instalación de speechbrain
BRANCH = 'develop'
!python -m pip install git+https://github.com/speechbrain/speechbrain.git@$BRANCH

In [ ]:
# Accede a un modelo pre-entrenado y utilízalo para transcribir los archivos de audio con los que has trabajado anteriormente
from speechbrain.inference.ASR import EncoderASR

#Descomenta este si quieres reconocer el audio en inglés:
# from speechbrain.inference.ASR import EncoderDecoderASR
# modelo_asr = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="pretrained_models/asr-crdnn-rnnlm-librispeech")
# modelo_asr.transcribe_file("migrabacion.wav")

modelo_asr = EncoderASR.from_hparams(source="speechbrain/asr-wav2vec2-commonvoice-14-es", savedir="pretrained_models/asr-wav2vec2-commonvoice-14-es")
modelo_asr.transcribe_file("miaudio_cortado.mp3")


# Viendo el sonido...

Existen distintas formas de visualizar el audio, vamos a ver algunas de las más importantes. [Aquí](https://speechprocessingbook.aalto.fi/Representations/Spectrogram_and_the_STFT.html) tienes material donde se explica cómo se interpretan.

Primero, vamos a importar las librerías necesarias:

In [ ]:
#importamos las bibliotecas necesarias
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

## Espectograma

In [ ]:
# Function to plot the spectrogram
def plot_spectrogram(audio_path):
    y, sr = librosa.load(audio_path)
    spect = librosa.feature.melspectrogram(y=y, sr=sr)
    spect_db = librosa.power_to_db(spect, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spect_db, x_axis='time', y_axis='mel', sr=sr, fmax=8000)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Espectograma')
    plt.show()

# Dibuja el espectograma
plot_spectrogram('./migrabacion.wav')

## Waveform

In [ ]:
def plot_waveform(audio_path):
    y, sr = librosa.load(audio_path)

    plt.figure(figsize=(12, 4))
    librosa.display.waveshow(y, sr=sr)
    plt.title('Waveform')
    plt.show()

# Dibuja el waveform
plot_waveform('./migrabacion.wav')

## Cromagrama

In [ ]:
def plot_cromograma(audio_path):
    y, sr = librosa.load(audio_path)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)

    plt.figure(figsize=(12, 4))
    librosa.display.specshow(chroma, y_axis='chroma', x_axis='time', sr=sr)
    plt.colorbar()
    plt.title('Cromagrama')
    plt.show()

# Dibuja el cromograma
plot_cromograma('./migrabacion.wav')


## Coeficientes cepstrales

In [ ]:
def plot_mfcc(audio_path):
    y, sr = librosa.load(audio_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    plt.figure(figsize=(12, 4))
    librosa.display.specshow(mfccs, x_axis='time', sr=sr)
    plt.colorbar()
    plt.title('MFCCs')
    plt.show()

# Dibuja los coeficientes cepstrales
plot_mfcc('./migrabacion.wav')